In [1]:
!pip install -q telebot

In [ ]:
!pip install -q opencv-python

In [2]:
SAMPLE_RATE = 48000

In [3]:
def resample_audio(path):
    audio, sr = librosa.load(path)
    if len(audio) < SAMPLE_RATE:
        audio = np.pad(audio, (1,48000-len(audio)), "constant")
    else:
        audio = audio[:SAMPLE_RATE]
    return audio



In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import librosa
from telebot import *
bot = telebot.TeleBot("6427139040:AAET4tZIh3hVkizhhiaSJ6ISN2DHbCFIdZ4")
model_Akhund_Human = tf.keras.models.load_model('/content/drive/MyDrive/dataset/Akhund_and_Human.h5')
model = tf.keras.models.load_model("/content/drive/MyDrive/Audio/audio.h5")
model_friends = tf.keras.models.load_model("/content/drive/MyDrive/Audio/audio_friends.h5")
label_names = ['Homayoun Shajaryan','Mohsen Chavoshi','Alireza Ghorbani','Siavash Ghomayshi']
label_names_friends = ['Parisa Baqerzade','Davood Fazeli','Javad Nematollahi','Khadijeh Valipour','kiana jhnshid','Rezaie','Matin Ghorbani','Mohammad','Mohammad_prf','Azra Khedadmand','Nima','Omid nomiri','Abdollah Ramezani','Shima Bazzazan','Maryam Saeedi','Sajedeh Gharabadiyan','Parsa']

flowers_name = ['daffodil','daisy', 'dandelion', 'fritillary', 'iris', 'lilyvalley', 'pansy', 'snowdrop', 'sunflower', 'tigerlily', 'tulip', 'windflower']


@bot.message_handler(commands=["start"])
def welcom(message):
    bot.reply_to(message, f"✋welcom  to {message.chat.first_name} boot AI ")
@bot.message_handler(commands=["help"])
def help(message):
	bot.send_message(message.chat.id,"/gray - Imaeg gray 🌅 \n/sketch - Photo Sketch 📸\n/flowers - photo Flower detection🌹🌿\n/akhund - distinguish mullah and human👳🏻‍♂️👨🏻\n/friend - Audio Classification")

@bot.message_handler(commands=['gray'])
def send_pencil_sketch(message):
    global bot_state
    bot.send_message(message.chat.id, "Send me an image 🌅")
    bot_state = 'gray'

@bot.message_handler(commands=['sketch'])
def send_pencil_sketch(message):
    global bot_state
    bot.send_message(message.chat.id, "Send me an image 🌅")
    bot_state = 'Sketch'

@bot.message_handler(commands=['flowers'])
def send_photo(message):
    msg = bot.send_message(message.chat.id,"Send me an image flower🌹")
    bot.register_next_step_handler(msg,photo)

@bot.message_handler(commands=['akhund'])
def send_photo_akhunb(message):
    msg = bot.send_message(message.chat.id,"Send me an image 👳🏻‍♂️👨🏻")
    bot.register_next_step_handler(msg,recognizer)

@bot.message_handler(commands=['singers'])
def send_photo_akhunb(message):
    msg = bot.send_message(message.chat.id,"Send me an Audio singers 🗣")
    bot.register_next_step_handler(msg,Singers)

@bot.message_handler(commands=['friends'])
def send_photo_akhunb(message):
    msg = bot.send_message(message.chat.id,"Send me an Audio friends 🗣")
    bot.register_next_step_handler(msg,friends)

def save_image(file_info, image_name=None):
    downloaded_file = bot.download_file(file_info.file_path)
    if image_name:
        image_path = os.path.join('photos', image_name + '.jpg')
    else:
        image_path = os.path.join(file_info.file_path)

    with open(image_path, 'wb') as new_file:
        new_file.write(downloaded_file)
    return image_path

@bot.message_handler(content_types=['photo'])
def send_photo(message):
    global bot_state, content_image_path
    file_info = bot.get_file(message.photo[-1].file_id)
    if bot_state == 'gray':
        image_path = save_image(file_info)
        image_result = image_gray(image_path)
    elif bot_state == 'Sketch':
        image_path = save_image(file_info)
        image_result = Sketch(image_path)

    image_path = os.path.join( file_info.file_path)
    cv2.imwrite(image_path, image_result)

    photo = open(image_path, "rb")
    bot.send_photo(message.chat.id, photo)

def image_gray(image_path):
    image = cv2.imread(image_path)
    image_result = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image_result
def dodge(x, y):
    return cv2.divide(x, 255 - y, scale=256)

def burn(image, mask):
    return 255 - cv2.divide(255 - image, 255 - mask, scale=256)

def Sketch(image_path):
    image = cv2.imread(image_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray_inv = 255 - image_gray
    image_gray_inv_blur = cv2.GaussianBlur(image_gray_inv, (21, 21), sigmaX=0, sigmaY=0)
    image_dodged = dodge(image_gray, image_gray_inv_blur)
    image_result = burn(image_dodged, image_gray_inv_blur)
    return image_result



@bot.message_handler(content_types = ["flowers"])
def photo(message):
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    downloaded_file = bot.download_file(file_info.file_path)

    with open("Flowers.jpg", 'wb') as new_file:
        new_file.write(downloaded_file)

    img = cv2.imread("Flowers.jpg")
    image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(image ,(224,224))
    img = img / 255
    img = img.reshape(1,224,224,3)
    result = np.argmax(model.predict(img))
    print(result)
    print(flowers_name[result])

    bot.send_message(message.chat.id,f' I feel your name is flowers {flowers_name[result]} ☘️🪴 .')


def recognizer(message):
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    downloaded_file = bot.download_file(file_info.file_path)
    width = height = 224


    with open("image.jpg", 'wb') as new_file:
        new_file.write(downloaded_file)

    img = cv2.imread("image.jpg")
    image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(image ,(width,height))
    img = img / 255
    img = img.reshape(1,width,height,3)

    result = np.argmax(model_Akhund_Human.predict(img))
    print(result)

    if result == 0:
        bot.send_message(message.chat.id,'از ما نیست👳🏻‍♂️')

    elif result == 1:
        bot.send_message(message.chat.id,'از ماست 👨🏻👩🏻')

def Singers(message):
    file_name = bot.get_file(message.audio.file_id)
    file_path = file_name.file_path
    file_download = bot.download_file(file_path)
    wav_filename = '/content/voice/output.wav'
    with open(wav_filename, 'wb') as audio:
      audio.write(file_download)

    asd = resample_audio(wav_filename)
    input_data  = asd.reshape(1,48000,1)
    # process
    preds = np.argmax(model(input_data))
    print(preds)
    print(label_names[preds])
    bot.reply_to(message,label_names[preds])

def friends(message):
  voice_name = bot.get_file(message.voice.file_id)
  file_download = bot.download_file(voice_name.file_path)
  file_path = voice_name.file_path

  with open(file_path, 'wb') as audio:
    audio.write(file_download)

  asd = resample_audio(file_path)
  input_data  = asd.reshape(1,48000,1)
  # process
  preds = np.argmax(model_friends(input_data))
  print(preds)
  print(label_names_friends[preds])
  bot.reply_to(message,label_names_friends[preds])


bot.infinity_polling()



1
Mohsen Chavoshi


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
)audio_info = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(audio_info.file_path)
    src = audio_info.file_path
    with open(src, 'wb') as audio_file:
        audio_file.write(downloaded_file)